# Query Cohere Embedding Models from AWS Bedrock 


Cohere builds a collection of Large Language Models (LLMs) trained on a massive corpus of curated web data. Powering these models, our infrastructure enables our product to be deployed for a wide range of use cases. The use cases we power include generation (copy writing, etc), summarization, classification, content moderation, information extraction, semantic search, and contextual entity extraction

This sample notebook shows you how to query [cohere-gpt-xlarge](https://aws.amazon.com/marketplace/pp/prodview-r6zvppobprqmy) using Amazon Bedrock.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Contents:
1. [Subscribe to Amazon Bedrock](#1.-Subscribe-to-Amazon-Bedrock)
2. [Perform real-time inference](#2.-Perform-real-time-inference)
   1. [Create a Cohere AWS Client](#A.-Create-a-Cohere-AWS-Client)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
3. [Sample use case](#3.-Sample-use-cases) 
   1. [Find the most and least similar texts](#A.-find-the-most-and-least-similar-texts)
4. [Clean up](#4.-Clean-up)
   1. [Unsubscribe from the listing (optional)](#A.-Unsubscribe-from-the-listing-(optional))

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to Amazon Bedrock

Follow the instructions in the [Amazon Bedrock](https://console.aws.amazon.com/bedrock) console.


## 2. Perform real-time inference

If you want to understand how real-time inference with Amazon Bedrock works, see the [Amazon Bedrock Documentation](https://docs.aws.amazon.com/bedrock).

### A. Create a Cohere AWS Client

In [1]:
%pip install --upgrade cohere-aws
# if you upgrade the package, you need to restart the kernel

import cohere_aws
import boto3

co = cohere_aws.Client(mode=cohere_aws.Mode.BEDROCK)

DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/tzdata-2023.3-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/schema-0.7.5-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/pytz-2023.3.post1-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/requests-2.31.0-py3.11.egg is deprecated. pip 23.3 will enforce this behaviour change. A possible replacement is to use pip for package installation..
DEPRECATION: Loading egg at /opt/homebrew/lib/python3.11/site-packages/ppft-1.7.6.7-py3.11.egg is deprecated. pip 2

### B. Create input payload

In [2]:
# Hacker News post titles
texts = ["Interesting (Non software) books?",
         "Non-tech books that have helped you grow professionally?",
         "I sold my company last month for $5m. What do I do with the money?",
         "How are you getting through (and back from) burning out?",
         "I made $24k over the last month. Now what?",
         "What kind of personal financial investment do you do?",
         "Should I quit the field of software development?"]
input_type = "clustering"
model_id = "cohere.embed-english-v3" # or "cohere.embed-multilingual-v3"

### C. Perform real-time inference

In [3]:
response = co.embed(texts=texts, input_type=input_type, model_id=model_id)

### D. Visualize output

In [10]:
for i in range(len(response.embeddings)):
    print(f"text: {texts[i]}")
    print(f"embedding: {response.embeddings[i]}\n")

text: Interesting (Non software) books?
embedding: [-0.023864746, -0.012016296, -0.033325195, 0.040527344, -0.06378174, -0.03845215, -0.00054740906, -0.011375427, -0.030395508, 0.000248909, -0.014503479, 0.035858154, -0.041381836, -0.059661865, 0.027282715, -0.017868042, 0.008659363, 0.01914978, 0.023010254, 0.012329102, -0.032104492, 0.012664795, 0.038330078, -0.11608887, -0.0048675537, -0.0023422241, 0.01550293, -0.03515625, 0.028564453, 0.005302429, 0.018722534, -0.03161621, 0.015029907, 0.025970459, -0.019073486, 0.0423584, 0.0004901886, -0.012786865, 0.020507812, -0.008560181, 0.018676758, 0.00037956238, -0.027954102, 0.016296387, -0.05810547, -0.012901306, -0.025619507, -0.030166626, 0.02960205, -0.010246277, 0.002281189, 0.03466797, 0.026519775, -0.0011882782, -0.015899658, -0.019958496, 0.005584717, -0.013336182, 0.016174316, -0.04559326, -0.021087646, 0.0056495667, 0.061920166, 0.0024547577, 0.00024032593, -0.018707275, 0.001709938, -0.003288269, -0.018554688, 0.0046577454, 0.

## 3. Sample use cases

### A. Find the most and least similar texts

In [17]:
from numpy import dot
from numpy.linalg import norm

def cosine_similarity(a, b):
    return dot(a, b)/(norm(a)*norm(b))

min_similarity = 1
min_similarity_texts = ()
max_similarity = 0
max_similarity_texts = ()

for i, emb1 in enumerate(response.embeddings[:-1]):
    for j, emb2 in enumerate(response.embeddings[i + 1:]):
        j = j + i + 1  # j's '0-index' is i + 1
        sim = cosine_similarity(emb1, emb2)
        if sim < min_similarity:
            min_similarity = sim
            min_similarity_texts = (texts[i], texts[j])
        if sim > max_similarity:
            max_similarity = sim
            max_similarity_texts = (texts[i], texts[j])

print(f"Most similar texts are {max_similarity_texts}")
print(f"Least similar texts are {min_similarity_texts}")

0
1
2
3
4
5
Most similar texts are ('Interesting (Non software) books?', 'Non-tech books that have helped you grow professionally?')
Least similar texts are ('Interesting (Non software) books?', 'I made $24k over the last month. Now what?')


## 4. Clean up

### A. Unsubscribe from the listing (optional)

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

